In [1]:
import sys
sys.path.append('../')

In [3]:
import pandas as pd

In [4]:
pm25 = pd.read_excel('../data/Bihar/Bihar_419_Sensor_PM25.xlsx', header=None)
rh = pd.read_excel('../data/Bihar/Bihar_419_Sensor_RH.xlsx', header=None)
temp = pd.read_excel('../data/Bihar/Bihar_419_Sensor_Temp.xlsx', header=None)

In [5]:
pm25 = pm25.tail(-1)
rh = rh.tail(-1)
temp = temp.tail(-1)

In [6]:
pm25

,0,1,2,3,4,5,6,7,8,9,...,482,483,484,485,486,487,488,489,490,491
1,Deviceid,PM25_1201230029,PM25_1201230033,PM25_1201230034,PM25_1201230052,PM25_1201230053,PM25_1201230074,PM25_1211220021,PM25_1212220153,PM25_1212220154,...,PM25_2CF4328C5AA7,PM25_3083988F9126,PM25_500291C0F7D2,PM25_68C63AD15069,PM25_8CAAB50AD70C,PM25_B4E62D2A3BA4,PM25_B4E62D2A46CD,PM25_BCDDC247BFE3,PM25_DC4F22364445,PM25_DC4F22364AC8
2,Block,ARARIA,KURSAKANTA,SIKTY,NARPATGANJ,BHARGAMA,RANIGANJ,FORBESGANJ,PALASI,JOKIHAT,...,BETTIAH,BAGAHA-I,CHANPATIA,BAIRIA,NAUTAN.W,LAURIYA,BAGAHA-II,BHITAHA,PIPRASI,MAINATAND
3,District,ARARIA,ARARIA,ARARIA,ARARIA,ARARIA,ARARIA,ARARIA,ARARIA,ARARIA,...,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN,WCHAMPARAN
4,lat,26.148,26.359,26.407,26.338,26.089,26.089,26.311,26.24,26.135,...,26.804,27.094,26.944213,26.754,26.704,26.994,27.133,26.907,27.06,27.102
5,long,87.457,87.443,87.549,87.107,87.152,87.243,87.245,87.565,87.615,...,84.52,84.093,84.537,84.448,84.479,84.41,84.06,84.136,84.023,84.647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,2023-08-18 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1900,2023-08-18 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1901,2023-08-18 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902,2023-08-18 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
pm25_devices = pm25.iloc[0, 1:].to_list()
rh_devices = rh.iloc[0, 1:].to_list()
temp_devices = temp.iloc[0, 1:].to_list()

pm25_block = pm25.iloc[1, 1:].to_list()
rh_block = rh.iloc[1, 1:].to_list()
temp_block = temp.iloc[1, 1:].to_list()

pm25_dist = pm25.iloc[2, 1:].to_list()
rh_dist = rh.iloc[2, 1:].to_list()
temp_dist = temp.iloc[2, 1:].to_list()

pm25_lat = pm25.iloc[3, 1:].to_list()
rh_lat = rh.iloc[3, 1:].to_list()
temp_lat = temp.iloc[3, 1:].to_list()

pm25_long = pm25.iloc[4, 1:].to_list()
rh_long = rh.iloc[4, 1:].to_list()
temp_long = temp.iloc[4, 1:].to_list()

pm25_time = pm25.iloc[7:, 0].to_list()
rh_time = rh.iloc[7:, 0].to_list()
temp_time = temp.iloc[7:, 0].to_list()

In [8]:
pm25_devices = [string.replace('PM25_', '').strip() for string in pm25_devices]
rh_devices = [string.replace('RH_', '').strip() for string in rh_devices]
temp_devices = [string.replace('Temp_', '').strip() for string in temp_devices]

In [9]:
def validData(pm25, rh, temp):
    assert (len(pm25) == len(rh) == len(temp)), "Same Number of Columns not Provided"

    for i in range(len(pm25)):
        assert(pm25[i] == rh[i] == temp[i]), f"{i}: {pm25[i]}, {rh[i]}, {temp[i]} \
            Data not provided in order"

In [10]:
validData(pm25_devices, rh_devices, temp_devices)
validData(pm25_block, rh_block, temp_block)
validData(pm25_dist, rh_dist, temp_dist)
validData(pm25_lat, rh_lat, temp_lat)
validData(pm25_long, rh_long, temp_long)
validData(pm25_time, rh_time, temp_time)

In [12]:
data = []
cols = pm25.columns

for i in range(7, len(pm25)):
    timestamp = pm25.iloc[i, 0]
    # To track the corresponding district, block ,device id, latitude, longitude for each column
    j = 0
    for col in range(1, len(cols)):
        row = {'timestamp': timestamp, 'device_id': pm25_devices[j], 'block': pm25_block[j], 'district': pm25_dist[j],\
            'latitude': pm25_lat[j], 'longitude': pm25_long[j], 'rh': rh.iloc[i, cols[col]], 'temp': temp.iloc[i, cols[col]],\
            'pm25': pm25.iloc[i, cols[col]],}
        j += 1
        data.append(row)

In [13]:
# data

In [14]:
df = pd.DataFrame(data)

In [15]:
df

,timestamp,device_id,block,district,latitude,longitude,rh,temp,pm25
0,2023-06-01 00:00:00,1201230029,ARARIA,ARARIA,26.148,87.457,NaN,NaN,NaN
1,2023-06-01 00:00:00,1201230033,KURSAKANTA,ARARIA,26.359,87.443,NaN,NaN,NaN
2,2023-06-01 00:00:00,1201230034,SIKTY,ARARIA,26.407,87.549,NaN,NaN,NaN
3,2023-06-01 00:00:00,1201230052,NARPATGANJ,ARARIA,26.338,87.107,NaN,NaN,NaN
4,2023-06-01 00:00:00,1201230053,BHARGAMA,ARARIA,26.089,87.152,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
930931,2023-08-18 23:00:00,B4E62D2A3BA4,LAURIYA,WCHAMPARAN,26.994,84.410,NaN,NaN,NaN
930932,2023-08-18 23:00:00,B4E62D2A46CD,BAGAHA-II,WCHAMPARAN,27.133,84.060,NaN,NaN,NaN
930933,2023-08-18 23:00:00,BCDDC247BFE3,BHITAHA,WCHAMPARAN,26.907,84.136,NaN,NaN,NaN
930934,2023-08-18 23:00:00,DC4F22364445,PIPRASI,WCHAMPARAN,27.060,84.023,NaN,NaN,NaN


In [16]:
df.to_pickle('../data/Bihar/Bihar_419_data.pkl')